## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import librosa

from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/음성 분류

/content/drive/MyDrive/음성 분류


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.2 MB/s eta 0:00:00


In [5]:
import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from IPython.display import Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

## Hyperparameter Setting

In [11]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # Melspectrogram 벡터를 추출할 개수
    'SEED':42
}

## Fixed Random-Seed

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing

In [42]:
train_df = pd.read_csv('/content/drive/MyDrive/음성 분류/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/음성 분류/test.csv')

In [43]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
### train data 음성 인식 진행하기

for train_num in range(0, 5001):
    file_name = f"/content/drive/MyDrive/음성 분류/train/TRAIN_{str(train_num).zfill(4)}.wav"

    data = wavfile.read(file_name)
    framerate = data[0]
    sounddata = data[1]
    time = np.arange(0,len(sounddata))/framerate

    input_audio, _ = librosa.load(file_name, sr=16000)

    input_values = tokenizer(input_audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    train_df["transcription"]=transcription

KeyboardInterrupt: ignored

In [ ]:
### test data 음성 인식 진행하기

for test_num in range(0, 1881):
    file_name = f"/content/drive/MyDrive/음성 분류/train/TRAIN_{str(test_num).zfill(4)}.wav"

    data = wavfile.read(file_name)
    framerate = data[0]
    sounddata = data[1]
    time = np.arange(0,len(sounddata))/framerate

    input_audio, _ = librosa.load(file_name, sr=16000)

    input_values = tokenizer(input_audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    test_df["transcription"]=transcription

In [ ]:
# file_name = '/content/drive/MyDrive/음성 분류/train/TRAIN_0000.wav' 
# Audio(file_name)

In [ ]:
# data = wavfile.read(file_name)
# framerate = data[0]
# sounddata = data[1]
# time = np.arange(0,len(sounddata))/framerate
# print(data)
# print('Sampling rate:',framerate,'Hz')

In [ ]:
# input_audio, _ = librosa.load(file_name, sr=16000)

In [ ]:
# input_values = tokenizer(input_audio, return_tensors="pt").input_values
# logits = model(input_values).logits
# predicted_ids = torch.argmax(logits, dim=-1)
# transcription = tokenizer.batch_decode(predicted_ids)[0]
# print(transcription)

In [ ]:
# def get_mfcc_feature(df):
#     features = []
#     for path in tqdm(df['path']):
#         # librosa패키지를 사용하여 wav 파일 load
#         y, sr = librosa.load(path, sr=CFG['SR'])
#         # librosa패키지를 사용하여 mfcc 추출
#         mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
#         y_feature = []
#         # 추출된 MFCC들의 평균을 Feature로 사용
#         for e in mfcc:
#             y_feature.append(np.mean(e))
#         features.append(y_feature)

#     mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
#     return mfcc_df

In [ ]:
# train_x = get_mfcc_feature(train_df)
# test_x = get_mfcc_feature(test_df)

In [ ]:
# train_y = train_df['label']

## Classification Model Fit

In [ ]:
model = DecisionTreeClassifier(random_state=CFG['SEED'])
model.fit(train_x, train_y)

## Inference

In [ ]:
preds = model.predict(test_x)

## Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = preds
submission.to_csv('./baseline_submission.csv', index=False)